In [1]:
import pandas as pd
# import glob
import datetime
start = datetime.datetime(2015, 8, 17)
end = datetime.datetime(2015, 9, 17)
dates = pd.date_range(start, end)
SOURCES = '_sources'
OUTPUT = '_output'
CURRENT_FOLDER = 'volkswagen_old' 
# 4 folders : volkswagen, vw, volkswagen_old, wv_old
# old = before scnadal

In [2]:
def get_date_splits(date_in):
    dates_out = 0
    if (type(date_in) == str):
        date_in = date_in.split('.000Z')
        dates_out = date_in[0].split('T')
    return dates_out

def get_date(date_in):
    dates_out = get_date_splits(date_in)
    return dates_out[0]

def get_time(date_in):
    dates_out = get_date_splits(date_in)
    return dates_out[1]

In [3]:
df_checks = pd.DataFrame(columns=['Date_start','Date_end', 'Time_start', 'Time_end','Nb_Tweets','Complete', 'Date_OK'])
nb_fichiers = 0
nb_observations = 0
index = 0
incomplets = []

for date in dates:
    date = date.strftime('%Y-%m-%d')
    file = '{}/{}/{}.json'.format(SOURCES, CURRENT_FOLDER, date)
    nb_fichiers += 1
    index += 1
    
    print('DATE FILE NAME {} : {}'.format(nb_fichiers, date))
    df = pd.DataFrame(pd.read_json(file))
    
    df = df.drop(df[df.time==0].index) # on supprime s'il n'y a pas de timestamp
    tweets = len(df.index)
    nb_observations += tweets
    
    df['date'] = df['time'].apply(lambda x: get_date(x))
    df['time_new'] = df['time'].apply(lambda x: get_time(x))
    
    heure_debut = df['time_new'].min()
    heure_fin = df['time_new'].max()
    date_debut = df['date'].min()
    date_fin = df['date'].max()
    
    # df.to_pickle('{}/good/{}.pickle'.format(FOLDER, date))
    
    if (heure_debut.split(':')[0] != '00'):
        print('SCRAPING INCOMPLET')
        incomplets.append(date)
        complet = 'NO'
    else:
        complet = 'YES'
    #    df.to_pickle('{}/good/{}.pickle'.format(FOLDER, date))
    if ((date != date_debut) or (date != date_fin)) :
        date_OK = 'NO'
        print('PROBLEME DE DATE')
    else:
        date_OK = 'YES'
    print('-----------------')
                                
    df_checks.loc[index] = [date_debut, date_fin, heure_debut, heure_fin, tweets, complet, date_OK]

print('///////////')
print('NB FICHIERS: {}'.format(nb_fichiers))
print('NB TWEETS: {}'.format(nb_observations))
print('///////////')

DATE FILE NAME 1 : 2015-08-17
-----------------
DATE FILE NAME 2 : 2015-08-18
-----------------
DATE FILE NAME 3 : 2015-08-19
-----------------
DATE FILE NAME 4 : 2015-08-20
-----------------
DATE FILE NAME 5 : 2015-08-21
-----------------
DATE FILE NAME 6 : 2015-08-22
-----------------
DATE FILE NAME 7 : 2015-08-23
-----------------
DATE FILE NAME 8 : 2015-08-24
-----------------
DATE FILE NAME 9 : 2015-08-25
-----------------
DATE FILE NAME 10 : 2015-08-26
-----------------
DATE FILE NAME 11 : 2015-08-27
-----------------
DATE FILE NAME 12 : 2015-08-28
-----------------
DATE FILE NAME 13 : 2015-08-29
-----------------
DATE FILE NAME 14 : 2015-08-30
-----------------
DATE FILE NAME 15 : 2015-08-31
-----------------
DATE FILE NAME 16 : 2015-09-01
-----------------
DATE FILE NAME 17 : 2015-09-02
-----------------
DATE FILE NAME 18 : 2015-09-03
-----------------
DATE FILE NAME 19 : 2015-09-04
-----------------
DATE FILE NAME 20 : 2015-09-05
-----------------
DATE FILE NAME 21 : 2015-09-0

In [4]:
incomplets

[]

In [5]:
df_checks.head()

,Date_start,Date_end,Time_start,Time_end,Nb_Tweets,Complete,Date_OK
1,2015-08-17,2015-08-17,00:00:09,23:59:35,1981,YES,YES
2,2015-08-18,2015-08-18,00:00:24,23:58:21,1737,YES,YES
3,2015-08-19,2015-08-19,00:02:03,23:59:52,1475,YES,YES
4,2015-08-20,2015-08-20,00:00:04,23:59:20,1528,YES,YES
5,2015-08-21,2015-08-21,00:00:04,23:59:15,1370,YES,YES


In [6]:
df_checks.to_excel('{}/checks_{}.xlsx'.format(SOURCES, CURRENT_FOLDER))

In [7]:
# GET data from JSON to DataFrame
first = True
for date in dates:
    date = date.strftime('%Y-%m-%d')
    file = '{}/{}/{}.json'.format(SOURCES, CURRENT_FOLDER, date)
    df_tempo = pd.DataFrame(pd.read_json(file))
    if (first):
        df_out = df_tempo
        first = False
    else:
        df_out = df_out.append(df_tempo, ignore_index=True)
df_out.head(2)

,favoriteCount,hashtags,id,images,isPinned,isReplyTo,isRetweet,quote,replyCount,retweetCount,screenName,text,time,urls,userMentions
0,0,[],633427986943860736,[],False,False,False,NaN,1,0,FowlerVW,Volkswagen Invented A Self-Driving Stroller An...,2015-08-17T23:59:35.000Z,[{'url': 'http://www.buzzfeed.com/abagg/babys-...,[]
1,0,"[{'hashtag': '2015', 'indices': [18, 23]}, {'h...",633427655807922176,[],False,False,False,NaN,0,0,BradReusing,Old exhaust off. #2015 #volkswagen #golf #tdi...,2015-08-17T23:58:16.000Z,[{'url': 'https://instagram.com/p/6gO6c1sN2X/'...,[]


In [8]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48085 entries, 0 to 48084
Data columns (total 15 columns):
favoriteCount    48085 non-null int64
hashtags         48085 non-null object
id               48085 non-null int64
images           48085 non-null object
isPinned         48085 non-null bool
isReplyTo        48085 non-null bool
isRetweet        48085 non-null bool
quote            252 non-null object
replyCount       48085 non-null int64
retweetCount     48085 non-null int64
screenName       48085 non-null object
text             48085 non-null object
time             48085 non-null object
urls             48085 non-null object
userMentions     48085 non-null object
dtypes: bool(3), int64(4), object(8)
memory usage: 4.5+ MB


In [9]:
df_out.to_pickle('{}/{}_total_brut.pickle'.format(OUTPUT,CURRENT_FOLDER))